In [1]:
from app import app


In [3]:
%pip install gunicorn
!pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip freeze | findstr "flask pymongo gunicorn" > requirements.txt



In [5]:
%%writefile requirements.txt
Flask
pymongo
gunicore

Writing requirements.txt


In [ ]:
from app import app# to import the Flask app object from app.py

if __name__ == "__main__":
    app.run()

 * Serving Flask app 'app'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [1]:
wsgi_content = """ from app import app

if __name__ == "__main__":
    app.run()
"""
with open("wsgi.py", "w") as f:
    f.write(wsgi_content)
print("wsgi.py created successfully")

wsgi.py created successfully


In [3]:
runtime_content = "python-3.10.12"

with open("runtime.txt", "w") as f:
    f.write(runtime_content)

print("runtime.txt created successfully")

runtime.txt created successfully


In [ ]:
# Flask app with MongoDB Storage + Validation + Success page

from flask import Flask, render_template, request
from pymongo import MongoClient
import os

app = Flask(__name__)

# --- MongoDB Connection ---
client = MongoClient("mongodb://localhost:27017/")
db = client["flask_health_care_app"]
users_collection = db["users"]

# --- Ensure template folder exists ---
templates_dir = r"C:\Users\jeffr\flask_health_care_app\templates"
os.makedirs(templates_dir, exist_ok=True)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":

        # basic user details
        try:
            # --- Basic user details ---
            age = int(request.form.get("age"))
            gender = request.form.get("gender")
            income = float(request.form.get("income"))
        except (ValueError, TypeError):
            return render_template(
                "form.html",
                error="Please enter valid numeric values for Age and Income."
            )

        # --- Expense Categories ---
        expense_categories = {
            "utilities": ("utilities_checkbox", "utilities_amount"),
            "entertainment": ("entertainment_checkbox", "entertainment_amount"),
            "school_fees": ("school_checkbox", "school_amount"),
            "shopping": ("shopping_checkbox", "shopping_amount"),
            "healthcare": ("healthcare_checkbox", "healthcare_amount"),
        }

        expenses = {}

        for category, (checkbox, amount_field) in expense_categories.items():
            selected = request.form.get(checkbox) == "on"
            amount_value = request.form.get(amount_field)

            # Validation: Prevent entering amount without checking checkbox
            
            if amount_value and not selected:
                return render_template(
                    "form.html",
                    error=f"You entered an amount for {category.replace('_', '').title()} without selecting the checkbox."
                )

            try:
                amount = float(amount_value) if amount_value else 0
            except ValueError:
                return render_template(
                    "form.html",
                    error=f"Invalid amount entered for {category.replace('_','').title()}."
                )

            expenses[category] = {
                "selected": selected,
                "amount": amount
            }

        # --- Build MongoDB Document ---
        user_data = {
            "age": age,
            "gender": gender,
            "total_income": income,
            "expenses": expenses
        }

        users_collection.insert_one(user_data)

        return render_template("success.html", user=user_data)

    return render_template("form.html")
     

# ------Run the Flask App -------
if __name__ == "__main__":
    app.run(port=5000, debug=True, use_reloader=False)


   

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Feb/2026 16:06:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2026 16:06:01] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2026 17:09:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2026 17:09:11] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Feb/2026 17:10:24] "POST / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "D:\jeffrey\Anaconda\envs\data_analytics_project\Lib\site-packages\flask\app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\jeffrey\Anaconda\envs\data_analytics_project\Lib\site-packages\flask\app.py", line 1514, in wsgi_app
    response = self.handle_exception(e)
  File "D:\jeffrey\Anaconda\envs\data_analytics_project\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\jeffrey\Anaconda\envs\data_analytics_project